# Step 0: Handling Imports and Dependencies

In [ ]:
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.2 MB/s eta 0:00:00
Reading package 

In [ ]:
import numpy as np
import pandas as pd
import re 

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Step 1: Preparing Dataset
Read the dataset into pandas dataframe

In [ ]:
data = pd.concat([pd.read_csv("/content/train.csv"),
                  pd.read_csv("/content/test.csv")],axis=0)


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89536 entries, 0 to 8127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweets  89534 non-null  object
 1   class   89527 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [ ]:
data.head()

,tweets,class
0,Be aware dirty step to get money #staylight ...,figurative
1,#sarcasm for #people who don't understand #diy...,figurative
2,@IminworkJeremy @medsingle #DailyMail readers ...,figurative
3,@wilw Why do I get the feeling you like games?...,figurative
4,-@TeacherArthurG @rweingarten You probably jus...,figurative


In [ ]:
np.unique(list(data["class"]))

array(['figurative', 'irony', 'nan', 'regular', 'sarcasm'], dtype='<U32')

In [ ]:
# remove nan values
data.dropna(inplace=True)
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 89527 entries, 0 to 8127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweets  89527 non-null  object
 1   class   89527 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


* Now it's okay. Let's write a class which will clean and vectorize our texts.

In [ ]:
class Vectorizer():
    def __init__(self,clean_pattern=None,max_features=None,stop_words=None):
        self.clean_pattern = clean_pattern
        self.max_features = max_features
        self.stopwords = stop_words
        self.tfidf = TfidfVectorizer(stop_words=self.stopwords,max_features=self.max_features)
        self.builded = False
        
    
    def _clean_texts(self,texts):
        
        cleaned = []
        for text in texts:
            if self.clean_pattern is not None:
                text = re.sub(self.clean_pattern," ",text)
            
            text = text.lower().strip()
            cleaned.append(text)
        
        return cleaned
    
    
    def _set_tfidf(self,cleaned_texts):
        self.tfidf.fit(cleaned_texts)
    
    def build_vectorizer(self,texts):
        cleaned_texts = self._clean_texts(texts)
        self._set_tfidf(cleaned_texts)
        self.builded = True
        
    def vectorizeTexts(self,texts):
        if self.builded:
            cleaned_texts = self._clean_texts(texts)
            return self.tfidf.transform(cleaned_texts)
        
        else:
            raise Exception("Vectorizer is not builded.")
            
            

* And let's create an object from this class and make our dataset cleaned and vectorized

In [ ]:
x = list(data["tweets"])
y = list(data["class"])

In [ ]:
vectorizer = Vectorizer("[^a-zA-Z0-9]",max_features=7000,stop_words="english");

In [ ]:
vectorizer.build_vectorizer(x)

In [ ]:
vectorized_x = vectorizer.vectorizeTexts(x).toarray()


In [ ]:
vectorized_x.shape

(89527, 7000)

* And now everything is okay with texts, let's encode the classes.

In [ ]:
label_map = {
    "figurative":0,
    "sarcasm":1,
    "irony":2,
    "regular":3
}


In [ ]:
y_encoded = []
for y_sample in y:
    y_encoded.append(label_map[y_sample])
    
y_encoded = np.asarray(y_encoded)

In [ ]:
y_encoded.shape

(89527,)

* And we're ready to create our custom Dataset object by inheriting it.

In [ ]:
class TweetDataset(Dataset):
    
    def __init__(self,x_vectorized,y_encoded):
        self.x_vectorized = x_vectorized
        self.y_encoded = y_encoded
        
    
    def __len__(self):
        return len(self.x_vectorized)
    
    
    def __getitem__(self,index):
        return self.x_vectorized[index],self.y_encoded[index]
    
    

* It's really easy to implement a custom dataset, let's create an object and test it.

In [ ]:
dataset = TweetDataset(vectorized_x,y_encoded)
print("Length of our dataset is",len(dataset))

print(dataset[2])

Length of our dataset is 89527
(array([0., 0., 0., ..., 0., 0., 0.]), 0)


* You know, to get random samples we need a random subset sampler, now we'll prepare it.

In [ ]:
# We've splitted our indices as train and test to use them in subset samplers.
train_indices,test_indices = train_test_split(list(range(0,len(dataset))),test_size=0.25,random_state=42)

In [ ]:
print(len(train_indices))
print(len(test_indices))

67145
22382


In [ ]:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)


* Our dataset and samplers are ready, we can create our data loader objects and start to model our artifical neural network.

In [ ]:
BATCH_SIZE = 128
train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,
                                                sampler=test_sampler)

# Step 2: Testing/Understanding the Transformer Architecture

In this section we're gonna create a custom network class which will be inherited from nn.Module and after creating our simple neural network we'll create loss function and optimizer.

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}